In [52]:
import os, sys
from numpy import inf
from time import time
from operator import itemgetter
from heapq import heappush, heappop, heapify

In [6]:
data_folder = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'data')
input_job_data = data_folder + "/job.txt"
input_edge_data = data_folder + "/edges.txt"

week 1: come up greedy algorithms from lecture for minimizing the weighted sum of completion times.
This file describes a set of jobs with positive and integral weights and lengths. It has the format

[number_of_jobs]

[job_1_weight] [job_1_length]

[job_2_weight] [job_2_length]

...

For example, the third line of the file is "74 59", indicating that the second job has weight 74 and length 59.

You should NOT assume that edge weights or lengths are distinct.


Q1: run greedy algorithm that schedules jobs in decreasing order of the difference (weight - length). IMPORTANT: if two jobs have equal difference (weight - length), you should schedule the job with higher weight first. 

Q2: run the greedy algorithm that schedules jobs (optimally) in decreasing order of the ratio (weight/length). In this algorithm, it does not matter how you break ties.


In [27]:
job_data = []
with open(input_job_data, 'r') as f_job:
    head_line = next(f_job)
    print("number_of_jobs :", head_line)
    for line in f_job:
        line_val = ([int(val) for val in line.split()])
        val_diff = line_val[0] - line_val[1]
        val_ratio = line_val[0]/line_val[1]
        line_val.extend([val_diff, val_ratio])
        job_data.append(line_val)
print("len of job_data", len(job_data), job_data[0:2])

number_of_jobs : 10000

len of job_data 10000 [[8, 50, -42, 0.16], [74, 59, 15, 1.2542372881355932]]


In [25]:
sort_by_diff_than_weight = sorted(job_data, key=itemgetter(2, 0), reverse=True)
complete_time = 0
result_time = 0
for job in sort_by_diff_than_weight:
    complete_time += job[1]
    result_time += job[0] * complete_time
print("total time required sort by diff:", result_time)

total time required sort by diff: 69119377652


In [26]:
sort_by_ratio = sorted(job_data, key=itemgetter(3), reverse=True)
complete_time = 0
result_time = 0
for job in sort_by_ratio:
    complete_time += job[1]
    result_time += job[0] * complete_time
print("total time required sort by ratio: ", result_time)

total time required sort by ratio:  67311454237


---------------------------------------------------------------------
Section II:
This file describes an undirected graph with integer edge costs. It has the format

[number_of_nodes] [number_of_edges]

[one_node_of_edge_1] [other_node_of_edge_1] [edge_1_cost]

[one_node_of_edge_2] [other_node_of_edge_2] [edge_2_cost]

...

For example, the third line of the file is "2 3 -8874", indicating that there is an edge connecting vertex #2 and vertex #3 that has cost -8874.

You should NOT assume that edge costs are positive, nor should you assume that they are distinct.

Q3: run Prim's minimum spanning tree algorithm on this graph. You should report the overall cost of a minimum spanning tree --- an integer, which may or may not be negative.
IMPLEMENTATION NOTES: This graph is small enough that the straightforward O(mn) time implementation of Prim's algorithm should work fine. 
OPTIONAL: For those of you seeking an additional challenge, try implementing a heap-based version.
    The simpler approach, which should already give you a healthy speed-up, is to maintain relevant edges in a heap (with keys = edge costs).
    The superior approach stores the unprocessed vertices in the heap, as described in lecture. Note this requires a heap that supports deletions, and you'll probably need to maintain some kind of mapping between vertices and their positions in the heap.

In [35]:
node_edges = {}
with open(input_edge_data, "r") as f_edges:
    header_line = next(f_edges)
    print("num of nodes and edges are corresponding:", head_line)
    for line in f_edges:
        line_vals = [int(val) for val in line.split()]
        node1, node2, edge_cost = line_vals[0], line_vals[1], line_vals[2]
        node_edges.setdefault(node1, []).append((node2, edge_cost))
        node_edges.setdefault(node2, []).append((node1, edge_cost))
print("double check node edge inputs:", len(node_edges))

num of nodes and edges are corresponding: 10000

double check node edge inputs: 500


In [68]:
#straightforward O(mn) time implementation of Prim's algorithm 
start_node = next(iter(node_edges))
explored_nodes = set([start_node])
tree_cost = 0
startTime = time()
while len(explored_nodes) != len(node_edges):
    min_cost, winner = inf, None
    for u in explored_nodes:
        for v, edge_cost in node_edges[u]:
            if (v not in explored_nodes) and (edge_cost <= min_cost):
                min_cost = edge_cost
                winner = v
    explored_nodes.add(winner)
    tree_cost += min_cost


print('Bad prims: ' + str(tree_cost) + ', ' + 'Time: ' + str(time() - startTime))

Bad prims: -3612829, Time: 0.1640021800994873


In [69]:
#speed up with heap implementation
start_node = next(iter(node_edges))
explored_nodes = set([start_node])
tree_cost = 0
startTime = time()
unexplored_nodes = []

for v, edge_cost in node_edges[start_node]:
    heappush(unexplored_nodes, (edge_cost, v))

while len(explored_nodes) != len(node_edges):
    min_cost, winner = heappop(unexplored_nodes)
    # add the winnder node to explored and mark it's adjacent edge case with explored node to inf
    explored_nodes.add(winner)
    tree_cost += min_cost
    
    for idx, edge_cost_node in enumerate(unexplored_nodes):
        if edge_cost_node[1] == winner:
            unexplored_nodes[idx] = (inf, winner)
    heapify(unexplored_nodes)
    
    for v, edge_cost in node_edges[winner]:
        if v not in explored_nodes:
            heappush(unexplored_nodes, (edge_cost, v))
print('He-basedap prims: ' + str(tree_cost) + ', ' + 'Time: ' + str(time() - startTime))

He-basedap prims: -3612829, Time: 0.14175105094909668
